In [10]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor
import math

logging.getLogger().setLevel(logging.INFO)

## The *Nim* and *Nimply* classes

In [11]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [12]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [13]:
def evaluate(strategyA: Callable, strategyB: Callable, num_matches = 1, nim_size = 3, k = None) -> float:
    players = (strategyA, strategyB)
    won = 0

    for _ in range(num_matches):
        nim = Nim(nim_size, k)
        player = 1
        while nim:
            ply = players[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return won / num_matches

## Random strategy

In [14]:
# Choose a random non empty row and remove a random number of objects smaller than min(k, row_objects)

def random_strategy(state: Nim):
    r = random.choice([idx for idx, r in enumerate(state.rows) if r > 0])
    num_objects = random.randint(1, min(state.rows[r], state.k) if state.k != None else state.rows[r])

    return (r, num_objects)

## Task 3.3: MinMax

In [15]:

def minmax(max_depth=math.inf):
    max_depth_reached = 0
    cache = {}
    hits = 0
    misses = 0
    MAX_CACHE_LEN = 1e6

    def minmax_with_pruning(state: Nim, alpha, beta, current_player, depth=0):
        nonlocal max_depth_reached, cache, hits, misses
        max_depth_reached = max(max_depth_reached, depth)
                
        if not state or depth >= max_depth:
            return current_player, None # i.e. the loser
        
        possible_moves = ((r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k)
        value = current_player * math.inf, None

        if current_player == 1: # my turn, minimize
            for m in possible_moves:
                tmp = deepcopy(state)
                tmp.nimming(m)
                
                s =  (tmp, -current_player, alpha, beta)

                if s not in cache or len(cache) >= MAX_CACHE_LEN:
                    val, _ = minmax_with_pruning(tmp, alpha, beta, -current_player, depth + 1)
                    cache.update({s: (val, None)}) 
                    misses += 1
                else:
                    val, _ = cache[s]
                    hits += 1
     
                value = min(value, (val, m))

                if value <= alpha:
                    break

                beta = min(beta, value)   
            return value
        else: # its turn, maximize
            for m in possible_moves:
                tmp = deepcopy(state)
                tmp.nimming(m)
                
                s =  (tmp, -current_player, alpha, beta)

                if s not in cache or len(cache) >= MAX_CACHE_LEN:
                    val, _ = minmax_with_pruning(tmp, alpha, beta, -current_player, depth + 1)
                    cache.update({s: (val, None)}) 
                    misses += 1
                else:
                    val, _ = cache[s]
                    hits += 1
                
                value = max(value, (val, m))
                
                if value >= beta:
                    break

                alpha = max(alpha, value)    
            return value

    def minmax_strategy_with_pruning(state: Nim):
        _, move = minmax_with_pruning(state, (-math.inf, None), (math.inf, None), 1)
        nonlocal max_depth_reached, hits, misses
        return move

    return minmax_strategy_with_pruning


## Oversimplified match

In [16]:
logging.getLogger().setLevel(logging.DEBUG)

strategy = (minmax(5) , random_strategy)

nim = Nim(7, None)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim) 
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13>
DEBUG:root:status: After player 0 -> <0 3 5 7 9 11 13>
DEBUG:root:status: After player 1 -> <0 2 5 7 9 11 13>
DEBUG:root:status: After player 0 -> <0 1 5 7 9 11 13>
DEBUG:root:status: After player 1 -> <0 1 5 7 3 11 13>
DEBUG:root:status: After player 0 -> <0 0 5 7 3 11 13>
DEBUG:root:status: After player 1 -> <0 0 5 7 3 4 13>
DEBUG:root:status: After player 0 -> <0 0 4 7 3 4 13>
DEBUG:root:status: After player 1 -> <0 0 4 5 3 4 13>
DEBUG:root:status: After player 0 -> <0 0 3 5 3 4 13>
DEBUG:root:status: After player 1 -> <0 0 3 5 3 0 13>
DEBUG:root:status: After player 0 -> <0 0 2 5 3 0 13>
DEBUG:root:status: After player 1 -> <0 0 0 5 3 0 13>
DEBUG:root:status: After player 0 -> <0 0 0 4 3 0 13>
DEBUG:root:status: After player 1 -> <0 0 0 4 0 0 13>
DEBUG:root:status: After player 0 -> <0 0 0 3 0 0 13>
DEBUG:root:status: After player 1 -> <0 0 0 2 0 0 13>
DEBUG:root:status: After player 0 -> <0 0 0 2 0 0 12>
DEBUG:root:status: Aft